In [4]:
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.base import BaseEstimator, TransformerMixin

class CustomScaler(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns, copy=True,with_mean=True, with_std=True):
        self.scaler = StandardScaler(copy,with_mean,with_std)
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    def transform(self, X, y=None, copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]
    
class absenteeism_model():
    
    def __init__(self, model_file, scaler_file):
        with open('model','rb') as model_file, open('scaler', 'rb') as scaler_file:
            self.reg = pickle.load(model_file)
            self.scaler = pickle.load(scaler_file)
            self.data = None
            
    def load_and_clean_data(self, data_file):
        data = pd.read_csv(data_file, delimiter=',')
        self.df_with_predictions = data.copy()
        data = data.drop(['ID'], axis=1)
    
        data['Absenteeism Time in Hours'] = 'NaN'
        
        reason_columns = pd.get_dummies(data['Reason for Absence'], drop_first=True)
        reason_type_1 = reason_columns.loc[:,1:14].max(axis=1)
        reason_type_2 = reason_columns.loc[:,15:17].max(axis=1)
        reason_type_3 = reason_columns.loc[:,18:21].max(axis=1)
        reason_type_4 = reason_columns.loc[:,22:].max(axis=1)
        
        data = data.drop(['Reason for Absence'], axis=1)
        data = pd.concat([data, reason_type_1, reason_type_2, reason_type_3, reason_type_4], axis=1)
        columns_names = ['Date', 'Transportation Expense', 'Distance to Work', 'Age', 
                         'Daily Work Load Average', 'Body Mass Index', 'Education', 
                         'Children', 'Pets', 'Absenteeism Time in Hours', 'Reason_1', 'Reason_2', 'Reason_3', 'Reason_4']
        
        data.columns = columns_names
        cols_ordered = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Date', 'Transportation Expense', 'Distance to Work', 'Age', 
                        'Daily Work Load Average', 'Body Mass Index', 'Education', 'Children', 'Pets', 'Absenteeism Time in Hours']
        
        data = data[cols_ordered]
        
        data.Date = pd.to_datetime(data.Date, format='%d/%m/%Y')
        
        list_months = []
        for i in range(data.shape[0]):
            list_months.append(data['Date'][i].month)
        data['Month Value'] = list_months
        
        day = []
        for i in range(data.shape[0]):
            day.append(data['Date'][i].dayofweek)
        data['Day of the Week'] = day
        
        data.drop(['Date'], axis=1, inplace=True)
        
        
        cols_upd = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value', 'Day of the Week', 
                    'Transportation Expense', 'Distance to Work', 'Age', 
                    'Daily Work Load Average', 'Body Mass Index', 'Education', 'Children', 
                    'Pets', 'Absenteeism Time in Hours']
        data = data[cols_upd]
        
        
        data['Education'] = data['Education'].map({1:0, 2:1, 3:1, 4:1})
        data = data.fillna(value=0)
        data.drop(['Absenteeism Time in Hours'], axis=1, inplace=True)
        data.drop(['Day of the Week', 'Distance to Work', 'Daily Work Load Average'], axis=1, inplace=True)
        
        self.preprocessed_data = data.copy()
        
        self.data = self.scaler.transform(data)
        
        
    def predicted_probability(self):
        if(self.data is not None):
            pred = self.reg.predict_proba(self,data)[:,1]
            return pred
        
    def predicted_cat(self):
        if(self.data is not None):
            pred_outputs = self.reg.predict(self.data)
            return pred_outputs
        
    def predicted_outputs(self):
        if(self.data is not None):
            self.preprocessed_data['Probability'] = self.reg.predict_proba(self.data)[:,1]
            self.preprocessed_data['Prediction'] = self.reg.predict(self.data)
            return self.preprocessed_data